In [1]:
pip install datasets transformers accelerate peft bitsandbytes


  Using cached accelerate-1.7.0-py3-none-any.whl.metadata (19 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
   ---------------------------------------- 0.0/10.5 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.5 MB ? eta -:--:--
   --- ------------------------------------ 0.8/10.5 MB 2.2 MB/s eta 0:00:05
   ----- ---------------------------------- 1.3/10.5 MB 2.4 MB/s eta 0:00:04
   -------- ------------------------------- 2.1/10.5 MB 2.8 MB/s eta 0:00:03
   ------------ --------------------------- 3.1/10.5 MB 3.3 MB/s eta 0:00:03
   ----------------- ---------------------- 4.5/10.5 MB 3.8 MB/s eta 0:00:02
   ---------------------- ----------------- 5.8/10.5 MB 4.3 MB/s eta 0:00:02
   ---------------------------- ----------- 7.3/10.5


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install huggingface_hub


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
from datasets import load_dataset

ds = load_dataset("MohamedSaeed-dev/python-text-to-code")

c:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\datasets--MohamedSaeed-dev--python-text-to-code. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating test split: 100%|██████████| 887/887 [00:00<00:00, 78824.26 exampl

In [4]:
# Check keys (splits)
print(ds)


DatasetDict({
    train: Dataset({
        features: ['text', 'code'],
        num_rows: 463
    })
    test: Dataset({
        features: ['text', 'code'],
        num_rows: 51
    })
    validation: Dataset({
        features: ['text', 'code'],
        num_rows: 60
    })
})


In [1]:
import json

files = [
    ("train.json", "train.jsonl"),
    ("test.json", "test.jsonl"),
    ("valid.json", "valid.jsonl"),
]

for input_file, output_file in files:
    with open(input_file, "r", encoding="utf-8") as f:
        lines = f.readlines()

    # Just copy line-by-line if they are already JSON objects
    with open(output_file, "w", encoding="utf-8") as f:
        for line in lines:
            line = line.strip()
            if line:  # Skip empty lines
                try:
                    json_obj = json.loads(line)  # Validate JSON
                    f.write(json.dumps(json_obj, ensure_ascii=False) + "\n")
                except json.JSONDecodeError as e:
                    print(f"❌ Error in line: {line[:80]}... → {e}")

    print(f"✅ Processed {input_file} → {output_file}")


✅ Processed train.json → train.jsonl
✅ Processed test.json → test.jsonl
✅ Processed valid.json → valid.jsonl


In [4]:
import json
from deep_translator import GoogleTranslator
from tqdm import tqdm

# === Step 1: Translate text ===

# File paths
input_file = "PHP Train.jsonl"            # English input
translated_file = "PHP Train_ms.jsonl"    # Intermediate file with both English & Malay
final_output_file = "PHP Train Data.jsonl"  # Final output: only Malay prompt + code

# Field names
source_field = "text"
translated_field = "text_ms"

translated_data = []

print("🔄 Translating English prompts to Malay...")
with open(input_file, "r", encoding="utf-8") as f:
    lines = f.readlines()

for line in tqdm(lines):
    item = json.loads(line)
    english_text = item.get(source_field, "")

    try:
        malay_text = GoogleTranslator(source="en", target="ms").translate(english_text)
    except Exception as e:
        print(f"⚠️ Translation error: {e}")
        malay_text = "[TRANSLATION_FAILED]"

    item[translated_field] = malay_text
    translated_data.append(item)

# Save intermediate JSONL with English + Malay
with open(translated_file, "w", encoding="utf-8") as f:
    for item in translated_data:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

print(f"✅ Translated data saved to: {translated_file}")

# === Step 2: Extract only Malay + Code ===

cleaned_data = []
for item in translated_data:
    cleaned_item = {
        "text": item.get("text_ms", "[NO_TRANSLATION]"),
        "code": item.get("code", "")
    }
    cleaned_data.append(cleaned_item)

# Save final simplified JSONL
with open(final_output_file, "w", encoding="utf-8") as f:
    for item in cleaned_data:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

print(f"✅ Final cleaned dataset saved to: {final_output_file}")


🔄 Translating English prompts to Malay...


  0%|          | 0/3087 [00:00<?, ?it/s]

 91%|█████████▏| 2820/3087 [1:36:49<50:39:35, 683.06s/it]

⚠️ Translation error: ("Connection broken: ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None)", ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None))


100%|██████████| 3087/3087 [1:42:07<00:00,  1.99s/it]    

✅ Translated data saved to: PHP Train_ms.jsonl
✅ Final cleaned dataset saved to: PHP Train Data.jsonl
